# Document Reader - Implantação

Preencha aqui com detalhes sobre a tarefa.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [15]:
%%writefile Model.py
import joblib
import pandas as pd
import numpy as np


class Model:
    
    def __init__(self):
        
        artifacts = joblib.load("/tmp/data/document_reader.joblib")
        self.model = artifacts["model"]
    

    def predict(self, X, feature_names, meta=None):
        feature_names_pipeline = ['doc_id', 'retriever_score', 'context', 'question']
        feature_names_qa = ['context', 'question']
        
        if feature_names != feature_names_pipeline and feature_names != feature_names_qa:
            raise ValueError(f'feature_names deve ser {feature_names_pipeline} ou {feature_names_qa}')
        
        df_input = pd.DataFrame(X,columns=feature_names)
        question = df_input['question'].iloc[0]
        topn_contexts = list(df_input['context'])
        df_pred = self.model.forward(question=question,topn_contexts=topn_contexts)
        
        if feature_names == feature_names_pipeline:
            df_input = df_input[['doc_id', 'retriever_score']] 
            df_input['index'] = df_input.index
            df_result = pd.merge(df_input, df_pred, on='index', how='outer')
            del df_result['index']
        else:
            df_result = df_pred
        return df_result.to_numpy()

Writing Model.py


In [13]:
# import numpy as np
# with open('result.npy', 'rb') as f:
#     X = np.load(f,allow_pickle=True)

# X = X[:10]
# #X.shape

In [22]:
# import pandas as pd
# df = pd.read_csv("df_squad_bert_chuncked.csv")
# n_lines = 10
# question = df['question'][0]
# contexts = df['context'][:n_lines]
# questions = [question]*n_lines

# df_small = pd.DataFrame({'context':contexts,'question':questions})
# X = df_small.to_numpy()

In [1]:
# from Model import Model
# import pandas as pd
# model = Model()
# result = model.predict(X,['doc_id', 'retriever_score', 'context', 'question'])
# result = model.predict(X,['context', 'question'])
# result